In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

import sys

# setting path
sys.path.append('..')

from evaluation import get_time_step_fleet
from utils import load_solution

In [2]:
solution = load_solution("../data/solution_example.json")
df = get_time_step_fleet(solution=solution, ts=1)

In [3]:
df.columns

Index(['datacenter_id', 'server_generation', 'server_id', 'action'], dtype='object')

In [4]:
from utils import load_problem_data
demand, datacenters, servers, selling_prices = load_problem_data('../data')

In [5]:
demand_selected = demand[demand['time_step'] == 1]
demand_selected

,time_step,latency_sensitivity,CPU.S1,CPU.S2,CPU.S3,CPU.S4,GPU.S1,GPU.S2,GPU.S3
0,1,high,4000,0,0,0,30,0,0
168,1,medium,6000,0,0,0,10,0,0
336,1,low,10000,0,0,0,10,0,0


In [6]:
datacenters

,datacenter_id,cost_of_energy,latency_sensitivity,slots_capacity
0,DC1,0.25,low,25245
1,DC2,0.35,medium,15300
2,DC3,0.65,high,7020
3,DC4,0.75,high,8280


In [7]:
merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')

# Calculate Total Revenue (assuming selling price represents revenue per time unit over life expectancy)
merged_servers_df['total_revenue'] = merged_servers_df['capacity'] * merged_servers_df['selling_price'] * merged_servers_df['life_expectancy']

# Calculate Total Costs
merged_servers_df['total_cost'] = (
    merged_servers_df['purchase_price'] +
    (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
    merged_servers_df['cost_of_moving']
)

# Calculate Profitability
merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']

# Sort by Profitability
most_profitable_servers = merged_servers_df.sort_values(by='profitability', ascending=False)

most_profitable_servers

,server_generation,latency_sensitivity,selling_price,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee,total_revenue,total_cost,profitability
20,GPU.S3,high,2700.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,2073600.0,456680,1616920.0
13,GPU.S3,medium,2450.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1881600.0,456680,1424920.0
6,GPU.S3,low,2150.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1651200.0,456680,1194520.0
19,GPU.S2,high,2000.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1536000.0,399720,1136280.0
18,GPU.S1,high,1875.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1440000.0,342760,1097240.0
12,GPU.S2,medium,1800.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1382400.0,399720,982680.0
11,GPU.S1,medium,1680.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1290240.0,342760,947480.0
5,GPU.S2,low,1600.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1228800.0,399720,829080.0
4,GPU.S1,low,1500.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1152000.0,342760,809240.0
17,CPU.S4,high,30.0,CPU,"[109,168]",22000,2,920,160,96,1000,423,460800.0,63608,397192.0


In [26]:
most_profitable_servers.to_csv("most_profitable_servers.csv")

In [8]:
most_profitable_servers

,server_generation,latency_sensitivity,selling_price,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee,total_revenue,total_cost,profitability
20,GPU.S3,high,2700.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,2073600.0,456680,1616920.0
13,GPU.S3,medium,2450.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1881600.0,456680,1424920.0
6,GPU.S3,low,2150.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1651200.0,456680,1194520.0
19,GPU.S2,high,2000.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1536000.0,399720,1136280.0
18,GPU.S1,high,1875.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1440000.0,342760,1097240.0
12,GPU.S2,medium,1800.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1382400.0,399720,982680.0
11,GPU.S1,medium,1680.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1290240.0,342760,947480.0
5,GPU.S2,low,1600.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1228800.0,399720,829080.0
4,GPU.S1,low,1500.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1152000.0,342760,809240.0
17,CPU.S4,high,30.0,CPU,"[109,168]",22000,2,920,160,96,1000,423,460800.0,63608,397192.0


In [24]:
demand['CPU.S1'].max()/60

11654.4

In [9]:
import uuid


def generate_random_fleet(datacenters=datacenters, servers=servers, num_servers=100, max_time_step=168):
    # Create lists of valid datacenter IDs and server generations
    datacenter_ids = datacenters['datacenter_id'].tolist()
    server_generations = servers['server_generation'].tolist()

    # Generate random data
    random_fleet = pd.DataFrame({
        'datacenter_id': np.random.choice(datacenter_ids, num_servers),
        'server_generation': np.random.choice(server_generations, num_servers),
        'server_id': [str(uuid.uuid4()) for _ in range(num_servers)],
        'time_step_of_purchase': np.random.randint(1, max_time_step + 1, num_servers)
    })

    return random_fleet

In [10]:
fleet = generate_random_fleet()
fleet

,datacenter_id,server_generation,server_id,time_step_of_purchase
0,DC1,GPU.S2,973ab9c7-cf74-44f4-ad24-51e67ee2fb00,8
1,DC3,CPU.S1,a3d4bcb5-f23b-4681-ad19-65c3542761e7,18
2,DC3,CPU.S1,f7f34c81-0cd1-47f6-afbf-eddf4333c948,20
3,DC3,GPU.S1,010a704a-2a9e-4cef-9eb5-5786e2d6e644,131
4,DC4,CPU.S1,8dcd3680-602d-48c7-b14b-e523f0495f52,23
...,...,...,...,...
95,DC3,CPU.S4,1b80871b-75d8-4099-b6c5-1be50415588f,47
96,DC2,GPU.S3,743a6107-2ba3-4269-bd94-1a7c753fd0e4,61
97,DC4,CPU.S2,a8d20830-f9bd-497d-b4a8-e5e37158fd6c,155
98,DC3,CPU.S4,221e08b0-646f-4499-b5b6-bf6b69049584,134


In [11]:
server_counts = fleet.groupby(['datacenter_id', 'server_generation']).size().reset_index(name="count")
server_counts

,datacenter_id,server_generation,count
0,DC1,CPU.S1,4
1,DC1,CPU.S2,1
2,DC1,CPU.S3,6
3,DC1,CPU.S4,2
4,DC1,GPU.S1,2
5,DC1,GPU.S2,2
6,DC1,GPU.S3,5
7,DC2,CPU.S1,4
8,DC2,CPU.S2,6
9,DC2,CPU.S3,3


In [12]:
fleet.loc[fleet["datacenter_id"] == "DC4"]

,datacenter_id,server_generation,server_id,time_step_of_purchase
4,DC4,CPU.S1,8dcd3680-602d-48c7-b14b-e523f0495f52,23
26,DC4,GPU.S3,44a2d27a-4863-4ec5-8eab-aa76bf15db77,134
35,DC4,GPU.S1,e55c5c13-98b5-49d3-9168-a902c2716afe,123
38,DC4,GPU.S3,16e0925c-534f-4489-98fc-5ff50d52de60,18
39,DC4,CPU.S4,68c55281-7b05-4767-8b36-1a74d7ec9238,101
40,DC4,CPU.S3,458cf462-4bf7-48a4-bc1c-9148b2ce29c9,38
47,DC4,CPU.S3,408e5664-0b07-4414-8ad1-baa45678866c,149
54,DC4,CPU.S1,a43a4d7f-2392-440d-b0fa-1626ead22be1,37
56,DC4,GPU.S1,5602ee44-ef67-4a8a-8bd4-7ccff597cdd5,41
58,DC4,GPU.S1,42b26f0b-4239-46b1-afa9-4180cce3f0b6,102


In [13]:
def get_server_ages(fleet, time_step):
    return fleet['time_step_of_purchase'].apply(lambda x: time_step - x)

In [14]:
get_server_ages(fleet, 168)

0     160
1     150
2     148
3      37
4     145
     ... 
95    121
96    107
97     13
98     34
99      8
Name: time_step_of_purchase, Length: 100, dtype: int64

In [28]:
# import pandas as pd
# from typing import List, Dict
# import uuid

# def load_and_preprocess_data():
#     # Load data
#     datacenters = pd.read_csv('../data/datacenters.csv')
#     servers = pd.read_csv('../data/servers.csv')
#     selling_prices = pd.read_csv('../data/selling_prices.csv')
#     demand = pd.read_csv('../data/demand.csv')

#     # Sort datacenters by cost_of_energy
#     datacenters = datacenters.sort_values('cost_of_energy')

#     return datacenters, servers, selling_prices, demand

# def calculate_server_profitability(servers: pd.DataFrame, selling_prices: pd.DataFrame) -> pd.DataFrame:
#     merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')
    
#     merged_servers_df['total_revenue'] = (
#         merged_servers_df['capacity'] * 
#         merged_servers_df['selling_price'] * 
#         merged_servers_df['life_expectancy']
#     )
    
#     merged_servers_df['total_cost'] = (
#         merged_servers_df['purchase_price'] +
#         (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
#         merged_servers_df['cost_of_moving']
#     )
    
#     merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']
    
#     return merged_servers_df.sort_values(by='profitability', ascending=False)

# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate total demand across all server types and latency sensitivities
#     total_demand = sum(current_demand[col] for col in current_demand.index if col not in ['time_step', 'latency_sensitivity'])
    
#     # Calculate current capacity
#     current_capacity = sum(server['capacity'] for server in current_fleet)
    
#     if total_demand > current_capacity:
#         # We need to add more servers
#         servers_to_add = total_demand - current_capacity
        
#         for _, datacenter in datacenters.iterrows():
#             if servers_to_add <= 0:
#                 break
            
#             available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
            
#             for _, server in servers.iterrows():
#                 if servers_to_add <= 0 or available_slots < server['slots_size']:
#                     break
                
#                 release_start, release_end = map(int, server['release_time'][1:-1].split(','))
#                 if release_start <= time_step <= release_end:
#                     # Server is available for purchase
#                     num_servers = min(servers_to_add // server['capacity'], available_slots // server['slots_size'])
                    
#                     for _ in range(num_servers):
#                         new_server = {
#                             'time_step': time_step,
#                             'datacenter_id': datacenter['datacenter_id'],
#                             'server_generation': server['server_generation'],
#                             'server_id': str(uuid.uuid4()),
#                             'action': 'buy',
#                             'capacity': server['capacity'],
#                             'slots_size': server['slots_size']
#                         }
#                         actions.append(new_server)
#                         current_fleet.append(new_server)
                        
#                         servers_to_add -= server['capacity']
#                         available_slots -= server['slots_size']
    
#     elif total_demand < current_capacity:
#         # We need to remove servers
#         servers_to_remove = current_capacity - total_demand
        
#         # Sort current fleet by datacenter cost (descending) and server age (descending)
#         current_fleet.sort(key=lambda x: (
#             -datacenters[datacenters['datacenter_id'] == x['datacenter_id']]['cost_of_energy'].iloc[0],
#             -x['time_step']
#         ))
        
#         for server in current_fleet:
#             if servers_to_remove <= 0:
#                 break
            
#             server_capacity = servers[servers['server_generation'] == server['server_generation']]['capacity'].iloc[0]
            
#             if servers_to_remove >= server_capacity:
#                 actions.append({
#                     'time_step': time_step,
#                     'datacenter_id': server['datacenter_id'],
#                     'server_generation': server['server_generation'],
#                     'server_id': server['server_id'],
#                     'action': 'dismiss'
#                 })
#                 current_fleet.remove(server)
#                 servers_to_remove -= server_capacity
    
#     return actions, current_fleet

# def run_simulation(datacenters: pd.DataFrame, 
#                    servers: pd.DataFrame, 
#                    selling_prices: pd.DataFrame, 
#                    demand: pd.DataFrame) -> List[Dict]:
#     profitable_servers = calculate_server_profitability(servers, selling_prices)
#     servers = servers.merge(profitable_servers[['server_generation', 'profitability']], on='server_generation')
#     servers = servers.sort_values('profitability', ascending=False)
    
#     all_actions = []
#     current_fleet = []
    
#     for time_step in range(1, 169):  # 168 time steps
#         actions, current_fleet = allocate_servers(time_step, demand, datacenters, servers, current_fleet)
#         all_actions.extend(actions)
    
#     return all_actions

# def main():
#     datacenters, servers, selling_prices, demand = load_and_preprocess_data()
#     solution = run_simulation(datacenters, servers, selling_prices, demand)
    
#     # Convert solution to DataFrame and save as JSON
#     solution_df = pd.DataFrame(solution)
#     solution_df.to_json('solution.json', orient='records', indent=2)

# if __name__ == "__main__":
#     main()

In [30]:
# import pandas as pd
# import numpy as np
# from typing import List, Dict
# import uuid

# def load_and_preprocess_data():
#     # Load data
#     datacenters = pd.read_csv('../data/datacenters.csv')
#     servers = pd.read_csv('../data/servers.csv')
#     selling_prices = pd.read_csv('../data/selling_prices.csv')
#     demand = pd.read_csv('../data/demand.csv')

#     # Sort datacenters by cost_of_energy
#     datacenters = datacenters.sort_values('cost_of_energy')

#     return datacenters, servers, selling_prices, demand

# def calculate_server_profitability(servers: pd.DataFrame, selling_prices: pd.DataFrame) -> pd.DataFrame:
#     merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')
    
#     merged_servers_df['total_revenue'] = (
#         merged_servers_df['capacity'] * 
#         merged_servers_df['selling_price'] * 
#         merged_servers_df['life_expectancy']
#     )
    
#     merged_servers_df['total_cost'] = (
#         merged_servers_df['purchase_price'] +
#         (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
#         merged_servers_df['cost_of_moving']
#     )
    
#     merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']
    
#     return merged_servers_df.sort_values(by='profitability', ascending=False)

# def calculate_demand(current_demand: pd.Series, servers: pd.DataFrame) -> Dict[str, int]:
#     demand_dict = {}
#     for server_type in ['CPU', 'GPU']:
#         for latency in ['high', 'medium', 'low']:
#             server_gens = servers[servers['server_type'] == server_type]['server_generation'].tolist()
#             demand = sum(current_demand[gen] for gen in server_gens if gen in current_demand.index)
#             capacity = servers[servers['server_type'] == server_type]['capacity'].iloc[0]
#             demand_dict[f"{server_type}_{latency}"] = int(np.ceil(demand / capacity))
#     return demand_dict

# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate demand for each server type and latency sensitivity
#     demand_dict = calculate_demand(current_demand, servers)
    
#     # Calculate current capacity
#     current_capacity = {f"{server['server_type']}_{server['latency_sensitivity']}": 0 for server in current_fleet}
#     for server in current_fleet:
#         current_capacity[f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
#     for server_type_latency, required_servers in demand_dict.items():
#         server_type, latency = server_type_latency.split('_')
#         servers_to_add = max(0, required_servers - current_capacity[server_type_latency])
#         servers_to_remove = max(0, current_capacity[server_type_latency] - required_servers)
        
#         if servers_to_add > 0:
#             # Add servers
#             for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
#                 if servers_to_add <= 0:
#                     break
                
#                 available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
#                 for _, server in servers[(servers['server_type'] == server_type) & (servers['profitability'] == servers['profitability'].max())].iterrows():
#                     if servers_to_add <= 0 or available_slots < server['slots_size']:
#                         break
                    
#                     release_start, release_end = map(int, server['release_time'][1:-1].split(','))
#                     if release_start <= time_step <= release_end:
#                         # Server is available for purchase
#                         num_servers = min(servers_to_add, available_slots // server['slots_size'])
                        
#                         for _ in range(num_servers):
#                             new_server = {
#                                 'time_step': time_step,
#                                 'datacenter_id': datacenter['datacenter_id'],
#                                 'server_generation': server['server_generation'],
#                                 'server_id': str(uuid.uuid4()),
#                                 'action': 'buy',
#                                 'capacity': server['capacity'],
#                                 'slots_size': server['slots_size'],
#                                 'server_type': server_type,
#                                 'latency_sensitivity': latency
#                             }
#                             actions.append(new_server)
#                             current_fleet.append(new_server)
                            
#                             servers_to_add -= 1
#                             available_slots -= server['slots_size']
        
#         elif servers_to_remove > 0:
#             # Remove servers
#             servers_to_remove_list = []
#             for server in sorted(current_fleet, key=lambda x: (x['time_step'], -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0])):
#                 if servers_to_remove <= 0:
#                     break
                
#                 if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
#                     actions.append({
#                         'time_step': time_step,
#                         'datacenter_id': server['datacenter_id'],
#                         'server_generation': server['server_generation'],
#                         'server_id': server['server_id'],
#                         'action': 'dismiss'
#                     })
#                     servers_to_remove_list.append(server)
#                     servers_to_remove -= 1
            
#             for server in servers_to_remove_list:
#                 current_fleet.remove(server)
    
#     return actions, current_fleet

# def run_simulation(datacenters: pd.DataFrame, 
#                    servers: pd.DataFrame, 
#                    selling_prices: pd.DataFrame, 
#                    demand: pd.DataFrame) -> List[Dict]:
#     profitable_servers = calculate_server_profitability(servers, selling_prices)
#     servers = servers.merge(profitable_servers[['server_generation', 'profitability']], on='server_generation')
#     servers = servers.sort_values('profitability', ascending=False)
    
#     all_actions = []
#     current_fleet = []
    
#     for time_step in range(1, 169):  # 168 time steps
#         actions, current_fleet = allocate_servers(time_step, demand, datacenters, servers, current_fleet)
#         all_actions.extend(actions)
    
#     return all_actions

# def main():
#     datacenters, servers, selling_prices, demand = load_and_preprocess_data()
#     solution = run_simulation(datacenters, servers, selling_prices, demand)
    
#     # Convert solution to DataFrame and save as JSON
#     solution_df = pd.DataFrame(solution)
#     solution_df.to_json('solution.json', orient='records', indent=2)

# if __name__ == "__main__":
#     main()

KeyError: 'CPU_high'

In [39]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import uuid

def load_and_preprocess_data():
    # Load data
    datacenters = pd.read_csv('../data/datacenters.csv')
    servers = pd.read_csv('../data/servers.csv')
    selling_prices = pd.read_csv('../data/selling_prices.csv')
    demand = pd.read_csv('../data/demand.csv')

    # Sort datacenters by cost_of_energy
    datacenters = datacenters.sort_values('cost_of_energy')

    return datacenters, servers, selling_prices, demand

def calculate_server_profitability(servers: pd.DataFrame, selling_prices: pd.DataFrame) -> pd.DataFrame:
    merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')
    
    merged_servers_df['total_revenue'] = (
        merged_servers_df['capacity'] * 
        merged_servers_df['selling_price'] * 
        merged_servers_df['life_expectancy']
    )
    
    merged_servers_df['total_cost'] = (
        merged_servers_df['purchase_price'] +
        (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
        merged_servers_df['cost_of_moving']
    )
    
    merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']
    
    return merged_servers_df.sort_values(by='profitability', ascending=False)

def calculate_demand(current_demand: pd.Series, servers: pd.DataFrame) -> Dict[str, int]:
    demand_dict = {}
    for server_type in ['CPU', 'GPU']:
        for latency in ['high', 'medium', 'low']:
            server_gens = servers[servers['server_type'] == server_type]['server_generation'].tolist()
            demand = sum(current_demand.get(gen, 0) for gen in server_gens)
            capacity = servers[servers['server_type'] == server_type]['capacity'].iloc[0]
            demand_dict[f"{server_type}_{latency}"] = int(np.ceil(demand / capacity))
    return demand_dict

def initialize_current_capacity():
    return {f"{server_type}_{latency}": 0 
            for server_type in ['CPU', 'GPU'] 
            for latency in ['high', 'medium', 'low']}


In [ ]:

# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate demand for each server type and latency sensitivity
#     demand_dict = calculate_demand(current_demand, servers)
    
#     # Calculate current capacity
#     current_capacity = initialize_current_capacity()
#     for server in current_fleet:
#         current_capacity[f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
#     for server_type_latency, required_servers in demand_dict.items():
#         server_type, latency = server_type_latency.split('_')
#         servers_to_add = max(0, required_servers - current_capacity[server_type_latency])
#         servers_to_remove = max(0, current_capacity[server_type_latency] - required_servers)
        
#         if servers_to_add > 0:
#             # Add servers
#             for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
#                 if servers_to_add <= 0:
#                     break
                
#                 available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
#                 for _, server in servers[(servers['server_type'] == server_type) & (servers['profitability'] == servers['profitability'].max())].iterrows():
#                     if servers_to_add <= 0 or available_slots < server['slots_size']:
#                         break
                    
#                     release_start, release_end = map(int, server['release_time'][1:-1].split(','))
#                     if release_start <= time_step <= release_end:
#                         # Server is available for purchase
#                         num_servers = min(servers_to_add, available_slots // server['slots_size'])
                        
#                         for _ in range(num_servers):
#                             new_server = {
#                                 'time_step': time_step,
#                                 'datacenter_id': datacenter['datacenter_id'],
#                                 'server_generation': server['server_generation'],
#                                 'server_id': str(uuid.uuid4()),
#                                 'action': 'buy',
#                                 'capacity': server['capacity'],
#                                 'slots_size': server['slots_size'],
#                                 'server_type': server_type,
#                                 'latency_sensitivity': latency
#                             }
#                             actions.append(new_server)
#                             current_fleet.append(new_server)
                            
#                             servers_to_add -= 1
#                             available_slots -= server['slots_size']
        
#         elif servers_to_remove > 0:
#             # Remove servers
#             servers_to_remove_list = []
#             for server in sorted(current_fleet, key=lambda x: (x['time_step'], -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0])):
#                 if servers_to_remove <= 0:
#                     break
                
#                 if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
#                     actions.append({
#                         'time_step': time_step,
#                         'datacenter_id': server['datacenter_id'],
#                         'server_generation': server['server_generation'],
#                         'server_id': server['server_id'],
#                         'action': 'dismiss'
#                     })
#                     servers_to_remove_list.append(server)
#                     servers_to_remove -= 1
            
#             for server in servers_to_remove_list:
#                 current_fleet.remove(server)
    
#     return actions, current_fleet

In [33]:
# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate demand for each server type and latency sensitivity
#     demand_dict = calculate_demand(current_demand, servers)
    
#     # Calculate current capacity
#     current_capacity = initialize_current_capacity()
#     for server in current_fleet:
#         current_capacity[f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
#     for server_type_latency, required_servers in demand_dict.items():
#         server_type, latency = server_type_latency.split('_')
#         servers_to_add = max(0, required_servers - current_capacity[server_type_latency])
#         servers_to_remove = max(0, current_capacity[server_type_latency] - required_servers)
        
#         if servers_to_add > 0:
#             # Add servers
#             for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
#                 if servers_to_add <= 0:
#                     break
                
#                 available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
#                 # Filter servers by type and availability at current time step
#                 available_servers = servers[(servers['server_type'] == server_type) & 
#                                             (servers['release_time'].apply(lambda x: eval(x)[0] <= time_step <= eval(x)[1]))]
                
#                 # Sort available servers by profitability
#                 available_servers = available_servers.sort_values('profitability', ascending=False)
                
#                 for _, server in available_servers.iterrows():
#                     if servers_to_add <= 0 or available_slots < server['slots_size']:
#                         break
                    
#                     # Server is available for purchase
#                     num_servers = min(servers_to_add, available_slots // server['slots_size'])
                    
#                     for _ in range(num_servers):
#                         # Create a base server dictionary for actions
#                         new_server_actions = {
#                             'time_step': time_step,
#                             'datacenter_id': datacenter['datacenter_id'],
#                             'server_generation': server['server_generation'],
#                             'server_id': str(uuid.uuid4()),
#                             'action': 'buy'
#                         }

#                         # Add the simplified version to actions
#                         actions.append(new_server_actions)

#                         # Create a more detailed server dictionary for current_fleet
#                         new_server_fleet = {
#                             **new_server_actions,  # start with the base attributes from new_server_actions
#                             'capacity': server['capacity'],
#                             'slots_size': server['slots_size'],
#                             'server_type': server_type,
#                             'latency_sensitivity': latency
#                         }

#                         # Add the detailed version to current_fleet
#                         current_fleet.append(new_server_fleet)
                        
#                         servers_to_add -= 1
#                         available_slots -= server['slots_size']
        
#         elif servers_to_remove > 0:
#             # Remove servers (this part remains the same)
#             servers_to_remove_list = []
#             for server in sorted(current_fleet, key=lambda x: (x['time_step'], -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0])):
#                 if servers_to_remove <= 0:
#                     break
                
#                 if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
#                     actions.append({
#                         'time_step': time_step,
#                         'datacenter_id': server['datacenter_id'],
#                         'server_generation': server['server_generation'],
#                         'server_id': server['server_id'],
#                         'action': 'dismiss'
#                     })
#                     servers_to_remove_list.append(server)
#                     servers_to_remove -= 1
            
#             for server in servers_to_remove_list:
#                 current_fleet.remove(server)
    
#     return actions, current_fleet

In [36]:
# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate demand for each server type and latency sensitivity
#     demand_dict = calculate_demand(current_demand, servers)
    
#     # Calculate current capacity
#     current_capacity = {datacenter['datacenter_id']: initialize_current_capacity() for _, datacenter in datacenters.iterrows()}
#     for server in current_fleet:
#         current_capacity[server['datacenter_id']][f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
#     for server_type_latency, required_servers in demand_dict.items():
#         server_type, latency = server_type_latency.split('_')
#         total_current_capacity = sum(capacity[server_type_latency] for capacity in current_capacity.values())
#         servers_to_add = max(0, required_servers - total_current_capacity)
#         servers_to_remove = max(0, total_current_capacity - required_servers)

        
#         if servers_to_add > 0:
#             # Add servers
#             for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
#                 if servers_to_add <= 0:
#                     break
                
#                 available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
#                 # Filter servers by type and availability at current time step
#                 available_servers = servers[(servers['server_type'] == server_type) & 
#                                             (servers['release_time'].apply(lambda x: eval(x)[0] <= time_step <= eval(x)[1]))]
                
#                 # Sort available servers by profitability
#                 available_servers = available_servers.sort_values('profitability', ascending=False)
                
#                 for _, server in available_servers.iterrows():
#                     if servers_to_add <= 0 or available_slots < server['slots_size']:
#                         break
                    
#                     # Server is available for purchase
#                     num_servers = min(servers_to_add, available_slots // server['slots_size'])
                    
#                     for _ in range(num_servers):
#                         # Create a base server dictionary for actions
#                         new_server_actions = {
#                             'time_step': time_step,
#                             'datacenter_id': datacenter['datacenter_id'],
#                             'server_generation': server['server_generation'],
#                             'server_id': str(uuid.uuid4()),
#                             'action': 'buy'
#                         }

#                         # Add the simplified version to actions
#                         actions.append(new_server_actions)

#                         # Create a more detailed server dictionary for current_fleet
#                         new_server_fleet = {
#                             **new_server_actions,  # start with the base attributes from new_server_actions
#                             'capacity': server['capacity'],
#                             'slots_size': server['slots_size'],
#                             'server_type': server_type,
#                             'latency_sensitivity': latency
#                         }

#                         # Add the detailed version to current_fleet
#                         current_fleet.append(new_server_fleet)
#                         current_capacity[datacenter['datacenter_id']][server_type_latency] += 1
                        
#                         servers_to_add -= 1
#                         available_slots -= server['slots_size']
        
#         elif servers_to_remove > 0:
#             # Remove servers
#             servers_to_remove_list = []
#             # Sort current fleet by time_step (oldest first) and then by inverse profitability
#             sorted_fleet = sorted(
#                 current_fleet,
#                 key=lambda x: (
#                     x['time_step'],
#                     -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0]
#                 )
#             )
#             for server in sorted_fleet:
#                 if servers_to_remove <= 0:
#                     break
                
#                 if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
#                     actions.append({
#                         'time_step': time_step,
#                         'datacenter_id': server['datacenter_id'],
#                         'server_generation': server['server_generation'],
#                         'server_id': server['server_id'],
#                         'action': 'dismiss'
#                     })
#                     servers_to_remove_list.append(server)
#                     servers_to_remove -= 1
#                     current_capacity[server['datacenter_id']][server_type_latency] -= 1
            
#             for server in servers_to_remove_list:
#                 current_fleet.remove(server)
    
#     return actions, current_fleet

In [40]:
def check_and_dismiss_old_servers(time_step: int, current_fleet: List[Dict], life_expectancy: int = 96) -> Tuple[List[Dict], List[Dict]]:
    actions = []
    servers_to_remove = []
    
    for server in current_fleet:
        server_age = time_step - server['time_step'] + 1
        if server_age == life_expectancy - 1:  # Dismiss at 95th time step
            actions.append({
                'time_step': time_step,
                'datacenter_id': server['datacenter_id'],
                'server_generation': server['server_generation'],
                'server_id': server['server_id'],
                'action': 'dismiss'
            })
            servers_to_remove.append(server)
    
    for server in servers_to_remove:
        current_fleet.remove(server)
    
    return actions, current_fleet

In [44]:
def allocate_servers(time_step: int, 
                     demand: pd.DataFrame, 
                     datacenters: pd.DataFrame, 
                     servers: pd.DataFrame, 
                     current_fleet: List[Dict]) -> List[Dict]:
    # First, check and dismiss old servers
    old_server_actions, current_fleet = check_and_dismiss_old_servers(time_step, current_fleet)
    
    actions = old_server_actions  # Start with actions from dismissing old servers
    
    # Get demand for the current time step 
    # TODO: MISTAKE IT PICKS ONLY THE FIRST RECORD WHICH IS FOR HIGH DEMAND
    current_demand = demand[demand['time_step'] == time_step].iloc[0] 
    
    # Calculate demand for each server type and latency sensitivity
    demand_dict = calculate_demand(current_demand, servers)
    
    # Calculate current capacity
    current_capacity = {datacenter['datacenter_id']: initialize_current_capacity() for _, datacenter in datacenters.iterrows()}
    for server in current_fleet:
        current_capacity[server['datacenter_id']][f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
    for server_type_latency, required_servers in demand_dict.items():
        server_type, latency = server_type_latency.split('_')
        total_current_capacity = sum(capacity[server_type_latency] for capacity in current_capacity.values())
        servers_to_add = max(0, required_servers - total_current_capacity)
        servers_to_remove = max(0, total_current_capacity - required_servers)
        
        if servers_to_add > 0:
            # Add servers
            for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
                if servers_to_add <= 0:
                    break
                
                available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
                # Filter servers by type and availability at current time step
                available_servers = servers[(servers['server_type'] == server_type) & 
                                            (servers['release_time'].apply(lambda x: eval(x)[0] <= time_step <= eval(x)[1]))]
                
                # Sort available servers by profitability
                available_servers = available_servers.sort_values('profitability', ascending=False)
                
                for _, server in available_servers.iterrows():
                    if servers_to_add <= 0 or available_slots < server['slots_size']:
                        break
                    
                    # Server is available for purchase
                    num_servers = min(servers_to_add, available_slots // server['slots_size'])
                    
                    for _ in range(num_servers):
                        # Create a base server dictionary for actions
                        new_server_actions = {
                            'time_step': time_step,
                            'datacenter_id': datacenter['datacenter_id'],
                            'server_generation': server['server_generation'],
                            'server_id': str(uuid.uuid4()),
                            'action': 'buy'
                        }

                        # Add the simplified version to actions
                        actions.append(new_server_actions)

                        # Create a more detailed server dictionary for current_fleet
                        new_server_fleet = {
                            **new_server_actions,  # start with the base attributes from new_server_actions
                            'capacity': server['capacity'],
                            'slots_size': server['slots_size'],
                            'server_type': server_type,
                            'latency_sensitivity': latency
                        }

                        # Add the detailed version to current_fleet
                        current_fleet.append(new_server_fleet)
                        current_capacity[datacenter['datacenter_id']][server_type_latency] += 1
                        
                        servers_to_add -= 1
                        available_slots -= server['slots_size']
        
        elif servers_to_remove > 0:
            # Remove servers
            servers_to_remove_list = []
            # Sort current fleet by time_step (oldest first) and then by inverse profitability
            sorted_fleet = sorted(
                current_fleet,
                key=lambda x: (
                    x['time_step'],
                    -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0]
                )
            )
            for server in sorted_fleet:
                if servers_to_remove <= 0:
                    break
                
                if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
                    actions.append({
                        'time_step': time_step,
                        'datacenter_id': server['datacenter_id'],
                        'server_generation': server['server_generation'],
                        'server_id': server['server_id'],
                        'action': 'dismiss'
                    })
                    servers_to_remove_list.append(server)
                    servers_to_remove -= 1
                    current_capacity[server['datacenter_id']][server_type_latency] -= 1
            
            for server in servers_to_remove_list:
                current_fleet.remove(server)
    
    return actions, current_fleet

In [42]:
def run_simulation(datacenters: pd.DataFrame, 
                   servers: pd.DataFrame, 
                   selling_prices: pd.DataFrame, 
                   demand: pd.DataFrame) -> List[Dict]:
    profitable_servers = calculate_server_profitability(servers, selling_prices)
    servers = servers.merge(profitable_servers[['server_generation', 'profitability']], on='server_generation')
    servers = servers.sort_values('profitability', ascending=False)
    
    all_actions = []
    current_fleet = []
    
    for time_step in range(1, 169):  # 168 time steps
        actions, current_fleet = allocate_servers(time_step, demand, datacenters, servers, current_fleet)
        all_actions.extend(actions)
    
    return all_actions

In [45]:
def main():
    datacenters, servers, selling_prices, demand = load_and_preprocess_data()
    solution = run_simulation(datacenters, servers, selling_prices, demand)
    
    # Convert solution to DataFrame and save as JSON
    solution_df = pd.DataFrame(solution)
    solution_df.to_json('solution.json', orient='records', indent=2)

if __name__ == "__main__":
    main()

In [46]:
def calculate_demand(current_demand: pd.Series, servers: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the demand for each server generation and latency sensitivity.
    
    Args:
        current_demand (pd.DataFrame): Current demand for each server generation.   
        servers (pd.DataFrame): DataFrame containing server information.
    
    Returns:
        pd.DataFrame: A DataFrame with the following columns:
        - server_generation (str): The generation of the server.
        - latency (str): Latency sensitivity (high, medium, or low).
        - demand (int): Calculated demand for the server generation and latency.
    """
    # Create a DataFrame from the current_demand Series
    demand_df = current_demand.reset_index()
    demand_df.columns = ['server_generation', 'demand']

    # Create a DataFrame with all combinations of server generations and latencies
    latencies = pd.DataFrame({'latency': ['high', 'medium', 'low']})
    demand_df = demand_df.merge(latencies, how='cross')

    # Ensure demand is an integer
    demand_df['demand'] = demand_df['demand'].astype(int)

    return demand_df[['server_generation', 'latency', 'demand']]

In [52]:
demand[demand['time_step'] == 1].iloc[0]

time_step                 1
latency_sensitivity    high
CPU.S1                 4000
CPU.S2                    0
CPU.S3                    0
CPU.S4                    0
GPU.S1                   30
GPU.S2                    0
GPU.S3                    0
Name: 0, dtype: object

In [56]:
def calculate_servers_needed(demand_df: pd.DataFrame, servers_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the number of servers needed to satisfy the demand for all server generations.

    Args:
        demand_df (pd.DataFrame): DataFrame containing demand data for all server generations.
        servers_info (pd.DataFrame): DataFrame containing server information, including capacity.

    Returns:
        pd.DataFrame: Input DataFrame with additional columns 
                      for the number of servers needed for each generation.
    """
    # List of all server generations
    server_generations = ['CPU.S1', 'CPU.S2', 'CPU.S3', 'CPU.S4', 'GPU.S1', 'GPU.S2', 'GPU.S3']

    # Create a dictionary to store server capacities
    server_capacities = servers_df.set_index('server_generation')['capacity'].to_dict()

    for generation in server_generations:
        # Calculate the number of servers needed for each generation
        servers_needed_col = f'{generation}_servers_needed'
        demand_df[servers_needed_col] = (demand_df[generation] / server_capacities[generation]).apply(np.ceil).astype(int)
    return demand_df

In [69]:
demand_with_servers_needed = calculate_servers_needed(demand, servers)
current_demand = demand_with_servers_needed.loc[demand_with_servers_needed['time_step'] == 1]

In [ ]:
current_demand

In [64]:
for _, latency_demand in current_demand.iterrows():
    print(latency_demand['latency_sensitivity'])
    servers_needed = latency_demand.iloc[9:]
    # servers_to_buy = np.maximum(0, servers_needed - )

high
medium
low


In [ ]:
# def calculate_server_profitability(servers: pd.DataFrame,
#                                    selling_prices: pd.DataFrame, 
#                                    datacenters: pd.DataFrame) -> pd.DataFrame:
#     """
#     Calculate the profitability of each server type 
#         based on its revenue and costs, including energy consumption.

#     Args:
#         servers (pd.DataFrame): DataFrame containing server information.
#         selling_prices (pd.DataFrame): DataFrame containing selling price information.
#         datacenters (pd.DataFrame): DataFrame containing datacenter information.

#     Returns:
#         pd.DataFrame: A DataFrame with the original server information plus the
#         following additional columns:
#         - total_revenue (float): Total expected revenue over the server's lifetime.
#         - total_cost (float): Total expected cost over the server's lifetime, including energy costs.
#         - profitability (float): Difference between total_revenue and total_cost.
#     """
#     # Merge server information with selling prices
#     servers_profit = pd.merge(selling_prices, servers, on='server_generation')
    
#     # Calculate average energy cost across all datacenters
#     avg_energy_cost = datacenters['cost_of_energy'].mean()
    
#     # Calculate total revenue
#     servers_profit['total_revenue'] = (
#         servers_profit['capacity'] * 
#         servers_profit['selling_price'] * 
#         servers_profit['life_expectancy']
#     )
    
#     # Calculate total cost, including energy consumption
#     servers_profit['total_cost'] = (
#         servers_profit['purchase_price'] +
#         (servers_profit['average_maintenance_fee'] * servers_profit['life_expectancy']) +
#         (servers_profit['energy_consumption'] * avg_energy_cost * servers_profit['life_expectancy'])
#     )
    
#     # Calculate profitability
#     servers_profit['profitability'] = servers_profit['total_revenue'] - servers_profit['total_cost']
    
#     return servers_profit

In [67]:
# Define all possible latency sensitivities and server generations
all_latencies = ['low', 'medium', 'high']
all_generations = ['CPU.S1', 'CPU.S2', 'CPU.S3', 'CPU.S4', 'GPU.S1', 'GPU.S2', 'GPU.S3']

# Create a full grid of all combinations
index = pd.MultiIndex.from_product([all_latencies, all_generations], 
                                    names=['latency_sensitivity', 'server_generation'])

srv_counts = pd.DataFrame(0, index=index, columns=['count']).unstack(level='server_generation', fill_value=0)

In [68]:
srv_counts

count                                          
server_generation   CPU.S1 CPU.S2 CPU.S3 CPU.S4 GPU.S1 GPU.S2 GPU.S3
latency_sensitivity                                                 
high                     0      0      0      0      0      0      0
low                      0      0      0      0      0      0      0
medium                   0      0      0      0      0      0      0

In [ ]:
if not state.fleet.empty:
        actual_counts = state.fleet.groupby(['latency_sensitivity', 'server_generation']).size().unstack(fill_value=0)
        srv_counts.update(actual_counts)